### Autores:  
Blanco García, Gabriel: gabriel.blanco@cunef.edu  
Ferrín Meilán, Michelle: michelle.ferrin@cunef.edu

## Colegio Universitario de Estudios Financieros
### Máster en Data Science para Finanzas
Madrid, diciembre de 2020

# Support Vector Machine

La idea original de estos modelos parte del clasificador de máximo margen. Para estos clasificadores, surge una sofisticación, conocida como Support Vector Classifiers, para mejorar la clasificación en casos en los que el clasificador marginal es insufuciente.   

Posteriormente, para afrontar problemas de difícil separabilidad, surgen los Support Vector Machine. Estos modelos emplean una función kernel, que normalmente es de tipo lineal, polinomial o radial, para calcular las posiciones que tomarían las observaciones de un espacio n-dimensional en el resultante de aplicar la transformcaión del kernel.

La clave está en que el espacio dimensional no se llega a transformar, puesto que esto generaría problemas de cómputo. Esto es conocido también como el kernel trick. Se calculan las posiciones en dicho espacio, y se separan las clases mediante un hiperplano. De este modo, un problema no separable se puede trasnformar en separable. Estos modelos  permiten cierto grado de error de clasificación en el entrenamiento para poder mejorar las predicciones  

Los principales parámetros del SVM son C y gamma

### Parámetro C
Actúa como parámetro de regularización del modelo. Controla el trade off entre la correcta clasificación de los elementos en el entrenamiento y la decisión de la maximización del margen. 
Para valores grandes de C, se aceptará un valor de margen pequeño si se casifican mejor las observaciones en train. Valores pequeños permitiran un margen mas grande, com una función de decisión más simple, a costa de menor precisión

###  Parámetro Gamma
Es el parámetro que controla la curvatura de la frontera de decisión, se emplea cuando se usa el kernel radial

A continuación construimos distintos SVM's. No empleamos validación cruzada porque el tiempo de cómputo se extiende demasiado, pero sería la mejor manera de hacerlo.

In [2]:
# Manipulacion
import pandas as pd
import numpy as np

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modelos
from sklearn.svm import SVC
from sklearn.svm import LinearSVC # Support Vector Machine

# Evaluación preliminar del modelo 
from sklearn import metrics 

# Cargar pipelines
import pickle 

# Ocultar warnings
import warnings
warnings.filterwarnings('ignore')

# Guardar los modelos
%run ../src/operar_modelos.ipynb

Leemos los datos

In [3]:
X_train = pd.read_csv('../data/train/X_train.csv', engine='python')
y_train = pd.read_csv('../data/train/y_train.csv', engine='python')

X_test = pd.read_csv('../data/test/X_test.csv', engine='python')
y_test = pd.read_csv('../data/test/y_test.csv', engine='python')

Cargamos los pipelines previamente guardados. Para ello, definimos una función, puesto que vamos a cargarlos en múltiples  notebooks

In [35]:
def cargar_pipelines():
    '''
    Esta función no lleva argumentos, porque hace siempre lo mismo, cargar 
    los Pipelines guardados en su carpeta. Es necesario asignar el resultado 
    de la función a un objeto para poder utilizarlo con los modelos.
    
    '''
    preprocesador = pickle.load(open('../pipelines/preprocesador.sav', 'rb'))
    
    return preprocesador

In [37]:
preprocesador = cargar_pipelines()

Generamos y entrenamos el SVM con kernel radial

In [24]:
svm = Pipeline(steps=[
    ('preprocesado', preprocesador),
    
    ('clasificador', SVC(random_state=1234,
                         kernel='rbf', # kernel radial
                         max_iter=3000))]) # límite 3000 iteraciones

In [25]:
svm.fit(X_train, y_train)

Pipeline(steps=[('preprocesado',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('escalador',
                                                                   StandardScaler())]),
                                                  Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'annual_inc_joint',
       'dti_joint', 'mort_acc'],
      dtype='object')),
                                                 ('categoricas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(fill_value='perdido',
                                                                                 strategy='constant')),
                      

In [26]:
svm.score(X_train, y_train) # 68.57% en train

0.685785787951564

In [29]:
svm.score(X_test, y_test) # 68.36% en train

0.6836183861789937

Guardamos el modelo, pese a ser un desastre en términos de precisión.

In [ ]:
guardar_modelo(svm, '../models/trained_models/svm_radial.sav')

Como deja bastante que desear, probamos el Linear SVC

In [31]:
linear_svc = Pipeline(steps=[
    ('preprocesado', preprocesador),
    
    ('clasificador', LinearSVC(random_state=1234,
                               C=1.5, # parametro de regularizacion
                               max_iter=3000))])

In [32]:
linear_svc.fit(X_train, y_train)

Pipeline(steps=[('preprocesado',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('escalador',
                                                                   StandardScaler())]),
                                                  Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'annual_inc_joint',
       'dti_joint', 'mort_acc'],
      dtype='object')),
                                                 ('categoricas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(fill_value='perdido',
                                                                                 strategy='constant')),
                      

In [7]:
linear_svc.score(X_train, y_train) # 78.61%

0.7861810078106406

In [33]:
linear_svc.score(X_test, y_test) # 78.44% 

0.7844416317787947

La precisión cae algo en test, pero no parece que sea debido a overfitting. Este modelo es bastante mejor que el radial, en terminos de accuracy. Habrá que ver su matriz de confusión normalizada, junto a la del restode modelos, dentro del notebook de selección de modelos

Guardamos el modelo

In [ ]:
guardar_modelo(linear_svc, '../models/trained_models/SVM_Lineal_Final.sav')